# 딥러닝 기반 상품 카테고리 자동 분류 서버 예

### 파일에서 학습 데이터를 읽는다.

In [3]:
from sklearn.externals import joblib
import sys

In [4]:
import json


In [59]:
x_text_list = []
y_text_list = []
enc = sys.getdefaultencoding()
with open("refined_category_dataset.dat",encoding=enc) as fin:
    for line in fin.readlines():
#         print (line)
        info = json.loads(line.strip())
        x_text_list.append((info['pid'],info['name']))
        y_text_list.append(info['cate'])
        

In [60]:
#joblib.dump(y_name_id_dict,"y_name_id_dict.dat")

### text 형식으로 되어 있는 카테고리 명을 숫자 id 형태로 변환한다.

In [61]:
y_name_id_dict = joblib.load("y_name_id_dict.dat")

In [62]:
print(y_name_id_dict)

{'여행/e쿠폰': 15, '반려동물': 4, '디지털': 10, '도서/문구': 8, '출산/육아': 16, '잡화': 14, '컴퓨터': 3, '뷰티': 0, '가구/인테리어': 11, '식품': 6, '건강': 7, '의류': 2, '자동차/공구': 1, '생필품/주방': 13, '취미': 5, '스포츠/레저': 9, '가전': 12}


In [63]:

y_name_set = set(y_text_list)
y_name_id_dict = dict(zip(y_name_set, range(len(y_name_set))))
# print(y_name_id_dict.items())
y_id_name_dict = dict(zip(range(len(y_name_set)),y_name_set))
y_list = [y_name_id_dict[x] for x in y_text_list]

### train test 분리하는 방법 

In [64]:
from sklearn.model_selection import train_test_split
X_train, X_test , y_train, y_test = train_test_split(x_text_list, y_list, test_size=0.2, random_state=42)


## 딥러닝 기반 text 분류에 필요한 모듈 로드

In [65]:
import os
import sys
import keras
import gensim
import keras.preprocessing.text
import numpy
import gensim
from numpy import array
from numpy import argmax
from keras.utils import to_categorical
import numpy as np

#### 모델 파일을 만약 만들었다면, 아래와 같이 로드 가능하다.

In [66]:
#word2vec = gensim.models.KeyedVectors.load_word2vec_format('/workspace/resources/11st_all_product_name.segment.0918.15w100e3min.model', binary=True)
#word2vec.init_sims(replace=True)

### text 데이터를 word-id 형태로 변환한다.

In [81]:
tmp_list=list(map(lambda i : i[1],X_train))
print(tmp_list)

['도그차일드 레오파드애견신발 노랑 강아지슈즈 애', '불스 방청윤활제 360ML 윤활방청제 방청유 윤활유 녹', '[쿠스쿠스파이]블루베리치즈파이 800g(100g x 8조각)', '파사바체 유리 계량컵/주방용품/계량컵/계량용컵/조리도구/계량용품', '제주 코코몽에코파크+메이즈랜드  제주도 관광지 2곳 패키지 입장권 할인 /기프트제주/제주 승마체험/제주 승마/제주승마할인/제주 승마장 추천/제주도 승마체험 가격말타는곳 말타기', 'BA537 발목 서포트 LP-954 압박밴드 - 발목보호대 무', '[중고]서버용 HDD SAS Seagate ST300MP0005 300GB/15K', '(투핫) [헬로키티]헬로키티 쿠킹컵 세트', '인테리어 스위치커버 그래픽스티커 파인애플 WBSS7170', '멋스러운 캐주얼 정장 서스펜더 멜빵 TE1005', '1200M [미코아이엔티] 아트조이 DIY 명화그리기 숲속의 작은집', '3M  PF 프라이버시 필터 (15.6W)', 'LD-PA5 인도 사우디아라비아 등 해외여행 아답타', '[착한스포츠]STAR 고무연식 야구공(12개입)', '다람쥐PMC-230 고양이장난감 고양이용장난감 캣장난감', 'PS무료배송 [랄라룹시 윈터드레스패션세트(506539)] (인형별매) 랄라룹시 윈터드레스 패션세트 인형옷 인..', '보브 듀얼 커버 CC크림 30ml 21호 라이트 톤업 커버', '허니버터링쿠키(요쿠르트)국내산쿠키 X2개 강아지과자', '태성 캐논정품후드 EW-54 (EF-M 18-55mm 전용 후드)', '가을!! 날라리 단가라 상하 -BK /16FHONEY016', '아둘 패밀리룩 단가라 니트 기모 가족티 스트라이프76', 'Casio 카시오키보드 SA-46/32건반/미니건반/키보드/전자키보드/SA46 (SA-46)', '참숯 옥황토 맥반석 찜질팩 DS3860H 타이머 뜸질기', '종이시디케이스(100매 CD보관 종이씨디케이스 CD수납', '코럴B화병A 인테리어화병 인테리어소품화병 디자인꽃', '[포그니 성인용 소변패드] 국산/천

In [67]:
#print(X_train)

sequence_tokenizer = keras.preprocessing.text.Tokenizer()
sequence_tokenizer.fit_on_texts(map(lambda i : i[1],X_train))
max_features = len(sequence_tokenizer.word_index)

print(max_features)
print(sequence_tokenizer)


28627


In [68]:
def texts_to_sequences2(d_list, tokenizer, maxlen=300):
    seq = tokenizer.texts_to_sequences(d_list)
    print('mean:', numpy.mean([len(x) for x in seq]))
    print('std:', numpy.std([len(x) for x in seq]))
    print('median:', numpy.median([len(x) for x in seq]))
    print('max:', numpy.max([len(x) for x in seq]))
    seq = keras.preprocessing.sequence.pad_sequences(seq, maxlen=maxlen, padding='post', truncating='post')
    return seq

In [69]:
from numpy import array
from numpy import argmax
from keras.utils import to_categorical

In [70]:
train = texts_to_sequences2(map(lambda i : i[1],X_train),sequence_tokenizer)
test = texts_to_sequences2(map(lambda i : i[1],X_test),sequence_tokenizer)

mean: 7.78808823529
std: 3.50519855834
median: 7.0
max: 28
mean: 4.41352941176
std: 3.05366595726
median: 4.0
max: 23


In [71]:
print(train[0])

[7175 7176  493 7177  290    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0  

#### word의 embedding 형태의 weight 를 초기화 한다. 

In [72]:
input_dim = train.shape[1]
print(input_dim)
input_tensor = keras.layers.Input(shape=(input_dim,), dtype='int32')


300


In [73]:
word_vec_dim = 100
not_ct = 0
weights = numpy.zeros((max_features + 1, word_vec_dim))
for word, index in sequence_tokenizer.word_index.items():
    if False:
        pass
#     if word in word2vec.vocab:
#         weights[index, :] = word2vec[word]
    else:
        not_ct+=1
        weights[index, :] = numpy.random.uniform(-0.25, 0.25, word_vec_dim)
# del word2vec
# del sequence_tokenizer
print (not_ct)

28627


####  학습할 레이러를 구성한다.

In [74]:
embedded = keras.layers.Embedding(input_dim=max_features + 1,
                                  output_dim=word_vec_dim, input_length=input_dim,
                                  weights=[weights],trainable=True)(input_tensor)
# embedded2 = keras.layers.Dropout(0.9)(embedded)
# embedded2 = embedded

In [75]:
tensors = []
for filter_length in [3, 5]:
    tensor = keras.layers.Convolution1D(nb_filter=50, filter_length=filter_length)(embedded)
    tensor = keras.layers.Activation('relu')(tensor)
    tensor = keras.layers.MaxPooling1D(pool_length=input_dim - filter_length + 1)(tensor)
    tensor = keras.layers.Flatten()(tensor)
    tensors.append(tensor)

c:\users\dh kim\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(kernel_size=3, filters=50)`
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\dh kim\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=298)`
  """
c:\users\dh kim\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(kernel_size=5, filters=50)`
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\dh kim\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=296)`
  """


In [76]:
#embedded = keras.layers.Dropout(0.5)(embedded)
output_tensor = keras.layers.merge(tensors, mode='concat', concat_axis=1)
# output_tensor = keras.layers.Dropout(0.5)(output_tensor) # 0.7312
output_tensor = keras.layers.Dropout(0.5)(output_tensor) 
output_tensor = keras.layers.Dense(len(set(y_list)), activation='softmax')(output_tensor)

#output = Dense(NUM_CLASSES, input_dim = hidden_dim_2, activation = "softmax")(pool_rnn) # See equations (6) and (7).

cnn = keras.models.Model(input_tensor, output_tensor)
cnn.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
print(cnn.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 300)           0                                            
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 300, 100)      2862800     input_3[0][0]                    
____________________________________________________________________________________________________
conv1d_3 (Conv1D)                (None, 298, 50)       15050       embedding_2[0][0]                
____________________________________________________________________________________________________
conv1d_4 (Conv1D)                (None, 296, 50)       25050       embedding_2[0][0]                
___________________________________________________________________________________________

c:\users\dh kim\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
c:\users\dh kim\appdata\local\programs\python\python35\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [77]:
#print(y_test)
print(np.asarray(to_categorical(y_test)))

[[ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]]


In [26]:
cnn.fit(train, np.asarray(to_categorical(y_train)), batch_size=60, nb_epoch=10,
        validation_data=(test, np.asarray(to_categorical(y_test))))

c:\users\dh kim\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 6800 samples, validate on 1700 samples
Epoch 1/10
6800/6800 [==============================] - 38s - loss: 2.8713 - acc: 0.0691 - val_loss: 2.8226 - val_acc: 0.0759
Epoch 2/10
6800/6800 [==============================] - 42s - loss: 2.8172 - acc: 0.0887 - val_loss: 2.8072 - val_acc: 0.0953
Epoch 3/10
6800/6800 [==============================] - 43s - loss: 2.7705 - acc: 0.1076 - val_loss: 2.7934 - val_acc: 0.1106
Epoch 4/10
6800/6800 [==============================] - 41s - loss: 2.7356 - acc: 0.1291 - val_loss: 2.7814 - val_acc: 0.1200
Epoch 5/10
6800/6800 [==============================] - 42s - loss: 2.7063 - acc: 0.1471 - val_loss: 2.7694 - val_acc: 0.1347
Epoch 6/10
6800/6800 [==============================] - 43s - loss: 2.6742 - acc: 0.1687 - val_loss: 2.7572 - val_acc: 0.1471
Epoch 7/10
6800/6800 [==============================] - 42s - loss: 2.6367 - acc: 0.1899 - val_loss: 2.7445 - val_acc: 0.1535
Epoch 8/10
6800/6800 [==============================] - 42s - loss: 2.

In [78]:
eval_x_text_list = []
with open("soma8_test_data.dat",encoding=enc) as fin:
    for line in fin.readlines():
        info = json.loads(line.strip())
        eval_x_text_list.append((info['pid'],info['name']))


In [82]:
del tmp_list
tmp_list=list(map(lambda i : i[1],eval_x_text_list))
print(tmp_list)

['acchousee 스프레이/기아/오피러스 /2006-2007/Z8/라이트메탈/주문제품(15일예상)', '[엑스포넷주식회사]코일탭_M162.0_18.6mm_1EA 절삭공구 작업공구 공구류 코일탭 탭 공구 빠른배송', '[오가닉맘]펭코남아목도리(MBW7MF02) (인천점)', 'TASCO INFOCUS 골프거리측정기 골프거리 측정 망원경', '(KNF)극동고성능해빙기/고성능전기해빙기/10kw', '(P)인두 플라이어 드라이버등 공구세트 수공구 공구', '[토이트론]실바니안패밀리-유치원학예회세트(3589)', '[누리]사이즈가 크고 튼튼한 뉴테크 좌식 바베큐그릴', '국내산 혀클리너 x 1개(낱개)혀 크리너/입냄새 제거/구강 청결제/여행용/혀간 칫솔 세트/치실/치약/백태', '(추천) 철사밴드_40mm__(통(100EA))/ 호스밴드 호스조임 조임밴드 조임쇠 파이프밴드 철사밴드 철밴드조임', '[핫트랙스] Black 1타스 - [Shachihata] 일본 사찌하타-Artline 유성매직(유성마', 'i1 꽃을든남자 헤나 헤어젤(슈퍼홀드) 500ml/a', '동방애왕뜸기+진영감생명쑥봉(50개)+코끼리라이터', '1300K [투플레이스] 마르니 장스탠드', '김명희 우리애들밥상 고소감자 부침가루', 'HS_905_이지접톱270 절단기 톱날 접이식톱 접톱 절삭', '[ETUDEHOUSE]에뛰드하우스 내 피부를 부탁해 로얄젤리 마스크20ml_1매', 'GOGO!테이스티 우유먼치 (25p)', '1p 지구 스펀지볼(10cm) 미니공 스폰지공 미니볼 스폰지볼 스펀지공 스펀지볼', '[베어파우](BEARPAW) GODETIA NAVY(womens) K460015GD-W', 'H]점멸식 LED 네온/블랙보드 BE127/홍보/메뉴판/광고', 'No350/AK-47에어건/BB탄총/비비탄총', 'S자 스틸 푸쉬업바/요가학원판매용 헬스용품점판매용', '(MST시원한 가야 알로에농장(1.5리터)x12페트 알로에음료 음료수 알로에주스 음료 과일음료', '[GRET](홍콩)홍콩 디즈니랜드

In [90]:
# pred_list = clf.predict(vectorizer.transform(map(lambda i : i[1],eval_x_text_list)))

sequence_tokenizer2 = keras.preprocessing.text.Tokenizer()
print(sequence_tokenizer2)
sequence_tokenizer2.fit_on_texts(map(lambda i : i[1],eval_x_text_list))
print(sequence_tokenizer2)
max_features = len(sequence_tokenizer2.word_index)
#print(max_features)
inputd= texts_to_sequences2(map(lambda i : i[1],eval_x_text_list),sequence_tokenizer2)

#print(inputd[1])
"""
sequence_tokenizer = keras.preprocessing.text.Tokenizer()
sequence_tokenizer.fit_on_texts(map(lambda i : i[1],X_train))
max_features = len(sequence_tokenizer.word_index)

print(max_features)
print(sequence_tokenizer)

train = texts_to_sequences2(map(lambda i : i[1],X_train),sequence_tokenizer)
test = texts_to_sequences2(map(lambda i : i[1],X_test),sequence_tokenizer)
"""
#cnn.fit(train, np.asarray(to_categorical(y_train)), batch_size=60, nb_epoch=10, validation_data=(test, np.asarray(to_categorical(y_test))))
pred = cnn.predict(inputd)
pred_list = [argmax(y) for y in pred]
print(pred_list[0:30])

mean: 7.91176470588
std: 3.33546720383
median: 7.0
max: 28
[10, 15, 10, 9, 8, 10, 10, 10, 15, 10, 10, 15, 15, 10, 15, 8, 15, 10, 10, 16, 15, 10, 15, 16, 10, 10, 10, 10, 9, 15]


In [80]:
import requests
name='test1'
nickname='test1nickname'
mode='test'
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list)),
         'name':name,'nickname':nickname,'mode':mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)
print (d.json())

{'msg': 'If you pull docker image before 2017-09-27 21:30,  pull your docker image again.', 'precision': 0.058823529411764705}


### CNN 으로 추출한 이미지 데이터 사용하기 
 * 이 부분은 각자 한번 해보도록 해요